# Week07 - Prediction using Decision Tree (with Hyperparameter Tuning)

# ICP05

## Introduction and Overview


In this notebook, we will reuse the Universal Bank dataset.

This time, we are developing a model to predict whether a customer will accept a personal loan offer. The dataset contains 5000 observations and 14 variables. The data is available on one of my GitHub repos.

## Install and import necessary packages

In [1]:
# You may need to install xgboost (it's not part of the sklearn package)
#!conda install xgboost 

In [2]:
# import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier

np.random.seed(1)

## Load data 

In [3]:
df = pd.read_csv('https://github.com/timcsmith/MIS536-Public/raw/master/Data/UniversalBank.csv')
df.head(5)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


## Explore the dataset

In [4]:
# Explore the dataset
# read the first row of the dataset 
print(df.head())
print(df.columns)
print(df.describe())
print(df.info())

   ID  Age  Experience  Income  ZIP Code  Family  CCAvg  Education  Mortgage  \
0   1   25           1      49     91107       4    1.6          1         0   
1   2   45          19      34     90089       3    1.5          1         0   
2   3   39          15      11     94720       1    1.0          1         0   
3   4   35           9     100     94112       1    2.7          2         0   
4   5   35           8      45     91330       4    1.0          2         0   

   Personal Loan  Securities Account  CD Account  Online  CreditCard  
0              0                   1           0       0           0  
1              0                   1           0       0           0  
2              0                   0           0       0           0  
3              0                   0           0       0           0  
4              0                   0           0       0           1  
Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education'

## Clean/transform data (where necessary)

In [5]:
# based on findings from data exploration, we need to clean up colum names, as there are some leading whitespace characters
df.columns = [s.strip() for s in df.columns] 
df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

Drop the columns we are not using as predictors (see previous notebooks -- we are given a subset of input variables to consider)

In [6]:
df = df.drop(columns=['ID', 'ZIP Code'])

In [7]:
# translation education categories into dummy vars
df = df.join(pd.get_dummies(df['Education'], prefix='Edu', drop_first=True))
df.drop('Education', axis=1, inplace = True)

df.head(3)

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Edu_2,Edu_3
0,25,1,49,4,1.6,0,0,1,0,0,0,0,0
1,45,19,34,3,1.5,0,0,1,0,0,0,0,0
2,39,15,11,1,1.0,0,0,0,0,0,0,0,0


## Split data intro training and validation sets

In [8]:
# construct datasets for analysis
target = 'Personal Loan'
predictors = list(df.columns)
predictors.remove(target)
X = df[predictors]
y = df[target]

In [9]:
# create the training set and the test set 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)

## Prediction with Decision Tree (using default parameters)



You can find details about SKLearm's DecisionTree classifier [here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

Create a decision tree using all of the default parameters

In [10]:
dtree=DecisionTreeClassifier()

Fit the model to the training data

In [11]:
_ = dtree.fit(X_train, y_train)

Review of the performance of the model on the validation/test data

In [12]:
y_pred = dtree.predict(X_test)

In [13]:
print(f"{'Model':^18}{'Score':^18}")
print("************************************")
print(f"{'>> Recall Score:':18}{recall_score(y_test, y_pred)}")
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred)}")

      Model             Score       
************************************
>> Recall Score:  0.9060402684563759
Accuracy Score:   0.9873333333333333
Precision Score:  0.9642857142857143
F1 Score:         0.9342560553633219


Save the recall result from this model

In [14]:
dtree_recall = recall_score(y_test, y_pred)

## Prediction with RandomForest (using default parameters)

Like all our classifiers, RandomeForestClassifier has a number of parameters that can be adjusted/tuned. In this example below, we simply accept the defaults. You may want to experiment with changing the defaul values and also use GridSearchCV to explore ranges of values.

* n_estimators: The number of trees in the forsest
    - A deeper tree might increase the performance, but also the complexity and chances to overfit.
    - The value must be an integer greater than 0. Default is 100.  
* max_depth: The maximum depth per tree. 
    - Deeper trees might increase the performance, but also the complexity and chances to overfit.
    - The value must be an integer greater than 0. Default is None, which allows the tree to grow without constraint.
* See the SciKit Learn documentation for more details. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


In [15]:
rforest = RandomForestClassifier()

In [16]:
_ = rforest.fit(X_train, y_train)

In [17]:
y_pred = rforest.predict(X_test)

In [18]:
print(f"{'Model':^18}{'Score':^18}")
print("************************************")
print(f"{'>> Recall Score:':18}{recall_score(y_test, y_pred)}")
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred)}")

      Model             Score       
************************************
>> Recall Score:  0.8456375838926175
Accuracy Score:   0.9833333333333333
Precision Score:  0.984375
F1 Score:         0.9097472924187726


Save the recall result from this model

In [19]:
rforest_recall = recall_score(y_test, y_pred)

## Prediction with ADABoost (using default parameters)

Like all our classifiers, ADABoostClassifier has a number of parameters that can be adjusted/tuned. In this example below, we simply accept the defaults. You may want to experiment with changing the defaul values and also use GridSearchCV to explore ranges of values.

* max_depth: The maximum depth per tree. 
    - A deeper tree might increase the performance, but also the complexity and chances to overfit.
    - The value must be an integer greater than 0. Default is None (meaning, the tree can grow to a point where all leaves have 1 observation).
* learning_rate: The learning rate determines the step size at each iteration while your model optimizes toward its objective. 
    - A low learning rate makes computation slower, and requires more rounds to achieve the same reduction in residual error as a model with a high learning rate. But it optimizes the chances to reach the best optimum.
    - Larger learning rates may not converge on a solution.
    - The value must be between 0 and 1. Default is 0.3.
* n_estimators: The number of trees in our ensemble. 
    - Equivalent to the number of boosting rounds.
    - The value must be an integer greater than 0. Default is 100.
* See the SciKit Learn documentation for more details. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

In [20]:
aboost = AdaBoostClassifier()

In [21]:
_ = aboost.fit(X_train, y_train)

In [22]:
y_pred = aboost.predict(X_test)

In [23]:
print(f"{'Model':^18}{'Score':^18}")
print("************************************")
print(f"{'>> Recall Score:':18}{recall_score(y_test, y_pred)}")
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred)}")

      Model             Score       
************************************
>> Recall Score:  0.7248322147651006
Accuracy Score:   0.9626666666666667
Precision Score:  0.8780487804878049
F1 Score:         0.7941176470588235


Save the recall result from this model

In [24]:
aboost_recall = recall_score(y_test, y_pred)

In [27]:
n_estimators = [int(x) for x in np.linspace(1, 1000, 5)]
n_estimators

[1, 250, 500, 750, 1000]

### Using ada boost with parameter tuning

In [64]:
#Defining parameters
n_estimators = [50,100,200,250,300,350,400]
learning_rate = np.arange(0.1,1.0,0.1)
random_seed=1
param_grid_random = { 'n_estimators': n_estimators,
                      'learning_rate': learning_rate,
                     }

In [65]:
#Initializing Random search
best_random_search_model = RandomizedSearchCV(
        estimator=AdaBoostClassifier(random_state=random_seed), 
        scoring='recall', 
        param_distributions=param_grid_random, 
        n_iter = 60, 
        cv=10, 
        verbose=0, 
        n_jobs = -1,
        random_state=random_seed
    )
best_random_search_Ada_model = best_random_search_model.fit(X_train,y_train)

In [66]:
#Finding Best parameters
random_search_best_Ada_params = best_random_search_Ada_model.best_params_
print('Best parameters found: ', random_search_best_Ada_params)

Best parameters found:  {'n_estimators': 200, 'learning_rate': 0.9}


In [67]:
#Printing the results
y_pred = best_random_search_Ada_model.predict(X_test)

print("************************************")
print(f"{'>>Recall Score:':18}{recall_score(y_test, y_pred,average='weighted')}")
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred,average='weighted')}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred,average='weighted')}")
print("************************************")

************************************
>>Recall Score:   0.966
Accuracy Score:   0.966
Precision Score:  0.9647129594898725
F1 Score:         0.9645990723614486
************************************


In [52]:
adapram=recall_score(y_test, y_pred,average='weighted')

### Analysis

The average='weighted' parameter specifies that the function should calculate the weighted average of the recall score, which takes into account the number of samples in each class. 

Accuracy measures the proportion of correct predictions out of all the predictions made. It is calculated as the number of true positives plus true negatives divided by the total number of examples.
Recall, on the other hand, measures the proportion of true positives out of all the actual positive examples. It is calculated as the number of true positives divided by the sum of true positives and false negatives.
If there are no false negatives, meaning that all positive examples are correctly identified, then recall will be equal to accuracy. In this case, all positive examples are correctly classified, and there are no negative examples that are mistakenly classified as positive, which means that accuracy and recall will be the same.
However, in most cases, recall will be lower than accuracy because there are usually false negatives. False negatives are instances that are actually positive, but are classified as negative. These instances are not captured by the model, which means that recall will be lower than accuracy.

## Prediction with GradientBoostingClassifier

Like all our classifiers, GradientBoostingClassifier has a number of parameters that can be adjusted/tuned. In this example below, we simply accept the defaults. You may want to experiment with changing the defaul values and also use GridSearchCV to explore ranges of values.

* max_depth: The maximum depth per tree. 
    - A deeper tree might increase the performance, but also the complexity and chances to overfit.
    - The value must be an integer greater than 0. Default is None (meaning, the tree can grow to a point where all leaves have 1 observation).
* learning_rate: The learning rate determines the step size at each iteration while your model optimizes toward its objective. 
    - A low learning rate makes computation slower, and requires more rounds to achieve the same reduction in residual error as a model with a high learning rate. But it optimizes the chances to reach the best optimum.
    - Larger learning rates may not converge on a solution.
    - The value must be between 0 and 1. Default is 0.3.
* n_estimators: The number of trees in our ensemble. 
    - Equivalent to the number of boosting rounds.
    - The value must be an integer greater than 0. Default is 100.
* See the SciKit Learn documentation for more details. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [29]:
gboost = GradientBoostingClassifier()

In [30]:
_ = gboost.fit(X_train, y_train)

In [31]:
y_pred = gboost.predict(X_test)

In [32]:
print(f"{'Model':^18}{'Score':^18}")
print("************************************")
print(f"{'>> Recall Score:':18}{recall_score(y_test, y_pred)}")
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred)}")

      Model             Score       
************************************
>> Recall Score:  0.8657718120805369
Accuracy Score:   0.9826666666666667
Precision Score:  0.9555555555555556
F1 Score:         0.9084507042253522


Save the recall result from this model

In [41]:
gboost_recall = recall_score(y_test, y_pred)

## Prediction with XGBoost

Like all our classifiers, XGBoost has a number of parameters that can be adjusted/tuned. In this example below, we simply accept the defaults. You may want to experiment with changing the defaul values and also use GridSearchCV to explore ranges of values.

* max_depth: The maximum depth per tree. 
    - A deeper tree might increase the performance, but also the complexity and chances to overfit.
    - The value must be an integer greater than 0. Default is 6.
* learning_rate: The learning rate determines the step size at each iteration while your model optimizes toward its objective. 
    - A low learning rate makes computation slower, and requires more rounds to achieve the same reduction in residual error as a model with a high learning rate. But it optimizes the chances to reach the best optimum.
    - The value must be between 0 and 1. Default is 0.3.
* n_estimators: The number of trees in our ensemble. 
    - Equivalent to the number of boosting rounds.
    - The value must be an integer greater than 0. Default is 100.
* colsample_bytree: Represents the fraction of columns to be randomly sampled for each tree. 
    - It might improve overfitting.
    - The value must be between 0 and 1. Default is 1.
* subsample: Represents the fraction of observations to be sampled for each tree. 
    - A lower values prevent overfitting but might lead to under-fitting.
    - The value must be between 0 and 1. Default is 1.
* See the XGBoost documentation for more details. https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn 

In [42]:
xgboost = XGBClassifier()

In [43]:
_ = xgboost.fit(X_train, y_train)

In [44]:
y_pred = xgboost.predict(X_test)

In [45]:
print(f"{'Model':^18}{'Score':^18}")
print("************************************")
print(f"{'>> Recall Score:':18}{recall_score(y_test, y_pred)}")
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred)}")

      Model             Score       
************************************
>> Recall Score:  0.8926174496644296
Accuracy Score:   0.9853333333333333
Precision Score:  0.9568345323741008
F1 Score:         0.9236111111111113


Save the recall result from this model

In [46]:
xgboost = recall_score(y_test, y_pred)

## Step 6: Summarize results    

As usual -- in this section you provide a recap your approach, results, and discussion of findings. 


In [58]:
print("Recall scores...")
print(f"{'Decision Tree:':18}{dtree_recall}")
print(f"{'Random Forest:':18}{rforest_recall}")
print(f"{'Ada Boosted Tree:':18}{aboost_recall}")
print(f"{'Gradient Tree:':18}{gboost_recall}")
print(f"{'XGBoost Tree:':18}{xgboost}")
print(f"{'Ada Boost param:':18}{adapram}")

Recall scores...
Decision Tree:    0.9060402684563759
Random Forest:    0.8456375838926175
Ada Boosted Tree: 0.7248322147651006
Gradient Tree:    0.7449664429530202
XGBoost Tree:     0.8926174496644296
Ada Boost param:  0.966


### Results

From the above tests, we had initially performed the decision tree, random forest and also the boosting techniques that we had covered, from the tests we got the results for our metrics and we had considered recall. While recall is a suitable metric to employ when false negatives are costly, precision is preferable when false positives carry a high cost. It is important to consider both metrics together, as they may entail trade-offs between them. A high recall score may lead to a lower precision score and vice versa. Therefore, the choice of metric should be determined by the specific context and objectives of the problem at hand.
Recall is a performance metric that is particularly valuable when identifying positive cases is more critical than avoiding false positives. Situations where false negatives carry a higher cost than false positives.

Not only recall, but we can see that the overall performance of our model has increased with adaboost parameter tuned version, as the performance metrics have significantly better results.

Though we had used only few parameters and iteration as our computing power isnt enought, However we managed to get good results.
We got the highest recall which is 0.966 for our Ada boost algorithm with parameter tuning and randomsearchCV. compared to the ada boost without parameter tuning which had 0.72, this is a significant improvement which we achieved with parameter tuning.
Not only ada boost, We Could further imporve the all the models, by implementing similar procedure, and tuning the paramters and doing the random search to find the best fit for our data, this will results in better metric scores. The more parameters we tune appropriately we achieve better results.